In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import urllib
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import datetime
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
folder_path = f'C:/Users/user/PycharmProjects/PersianChildhood'
driver_path = 'C:/Users/User/Downloads/chromedriver_win32 (3)/chromedriver.exe'
page_url = f'https://digitale-sammlungen.ulb.uni-bonn.de/ulbbnioa/periodical/pageview'
base_url = 'https://digitale-sammlungen.ulb.uni-bonn.de/topic/view/3080607?max=30'
x = "?/:\\"
y = '----'
translate_dict = str.maketrans(x, y)

In [3]:
def scrape_pages(driver, page_url, new_path, volume_name, journal_name):
    x = "?/:"
    y = '---'
    translate_dict = str.maketrans(x, y)    
    page_numbers = []
    page_names = []
    page_links = []
    ls = driver.find_element_by_id('goToPage').find_elements_by_tag_name('option')
    for i in ls:
        page_numbers.append(i.get_attribute(name='value'))
        page_names.append(i.text.split()[-1])
    #for each page, open it, get the name of the magazine, and then download the full image
    for ind in range(len(ls)):
        page_link = f'{page_url}/{page_numbers[ind]}'
        driver.get(page_link)
        page_links.append(page_link)
        time.sleep(3)
        new_url = driver.find_element_by_class_name('singlePageImage').get_attribute(name='src')
        
        temp_path = f"{journal_name}/{volume_name}"
        if len(driver.find_elements_by_class_name('npNext'))>2:# there is at least one sub-volume
            for element in driver.find_elements_by_xpath("//*[@id='navPath']/div/a[contains(@href, 'pageview')]")[2:]:
                sub_volume = element.text.translate(translate_dict).rstrip(".").strip()
                temp_path = f"{temp_path}/{sub_volume}"
        mag_name = driver.find_element_by_id('navPath').find_elements_by_xpath('div')[-2].text.translate(translate_dict).rstrip(".").strip()        
        if len(mag_name.split('\n'))>1:
            mag_name = mag_name.split('\n')[1]
        temp_path = f"{temp_path}/{mag_name}"
        try:
            os.makedirs(f"{folder_path}/{temp_path}", exist_ok=False)
            print(f"creating folder: {folder_path}/{temp_path}")
        except:
            pass
        if os.path.isfile(f"{temp_path}/{page_names[ind]}.jpg"): # the file has already been downloaded
            print(f"the file has been downloaded already:{temp_path}/{page_names[ind]}.jpg")
            continue
        else:
            urllib.request.urlretrieve(new_url,f"{temp_path}/{page_names[ind]}.jpg")
            print(f"creating file: {temp_path}/{page_names[ind]}.jpg")

    return pd.DataFrame({'journal':journal_name,'volume':volume_name,'issue':mag_name,'page':page_names,'link':page_links,'extraction date':datetime.date.today()})


In [ ]:
#get all journals on a base_url page

#open base url
chrome = webdriver.ChromeOptions()
driver = webdriver.Chrome(driver_path, options=chrome) #ChromeDriverManager().install()
driver.get(base_url) 
#get list of all journal links
meta_data_dfs = []
journals = driver.find_elements_by_class_name('title')
journal_count = len(journals)

for count in range(journal_count):    
    driver.get(base_url) 
    #go to journal homepage, get title and create directory   
    journals = driver.find_elements_by_class_name('title')
    journal = journals[count]
    if len(journal.text.split('\n'))>1:
        journal_name = journal.text.split('\n')[1]
    journal_name = journal.text
    #journal_name = journal_name.replace(':','-')
    journal_name = journal_name.translate(translate_dict)
    journal.click()
    time.sleep(3)
    volume_elements = [element for element in driver.find_elements_by_class_name('childLink')]
    if len(volume_elements)>0: #there are multiple volumns
        volume_links = [element.get_attribute(name='href') for element in driver.find_elements_by_class_name('childLink')]
        volume_names = [element.text for element in driver.find_elements_by_class_name('partInfo')]
        for i in range(len(volume_links)):
            volume_name = volume_names[i].translate(translate_dict).strip()
            if len(volume_name.split('\n'))>1:
                volume_name = volume_name.split('\n')[1]
            new_path = f"{journal_name}/{volume_name}"
            # check if new path exists, if so, skip to next volume
            try:
                os.makedirs(f"{folder_path}/{new_path}", exist_ok=False)
                print(f"creating folder: {folder_path}/{new_path}")
            except:
                print(f"this folder already exists: {folder_path}/{new_path}")
                continue
            driver.get(volume_links[i])
            driver.find_element_by_class_name('imgLink').click()           
            meta_data_df = scrape_pages(driver, page_url, new_path, volume_name, journal_name)
            meta_data_dfs.append(meta_data_df)
            
    else: #no apparent additional volumes (but there still could be hidden volumes)
        driver.find_element_by_class_name('imgLink').click()
        #get potential volume(s) name(s) from dropdown menu
        try:
            driver.find_element_by_id('open-pageviewStructure-tree').click()
        except: #try again after refreshing
            driver.refresh()
            time.sleep(3)
            driver.find_element_by_id('open-pageviewStructure-tree').click()  
        volume_links = [volume.find_element_by_class_name('pagelink').get_attribute('href') for volume in driver.find_elements_by_class_name('journal-volume')]
        volume_names = [volume.find_element_by_class_name('pagelink').text for volume in driver.find_elements_by_class_name('journal-volume')]
        for i in range(len(volume_links)):
            volume_name = volume_names[i].translate(translate_dict).strip()
            if len(volume_name.split('\n'))>1:
                volume_name = volume_name.split('\n')[1]
            new_path = f"{journal_name}/{volume_name}"
            # check if new path exists, if so, skip to next volume
            try:
                os.makedirs(f"{folder_path}/{new_path}", exist_ok=False)
                print(f"creating folder: {folder_path}/{new_path}")
            except:
                print(f"this folder already exists: {folder_path}/{new_path}")
                continue
            driver.get(volume_links[i])
            meta_data_df = scrape_pages(driver, page_url, new_path, volume_name, journal_name)
            meta_data_dfs.append(meta_data_df)
            
#         volume_name = driver.find_element_by_class_name('journal_volume').text
#         volume_name = volume_name.translate(translate_dict) #remove unformatted char        
#         new_path = f"{journal_name}/{volume_name}"
#         # check if new path exists, if so, skip to next volume
#         try:
#             os.makedirs(f"{folder_path}/{new_path}", exist_ok=False)
#             print(f"creating folder: {folder_path}/{new_path}")
#         except:
#             print(f"this folder already exists: {folder_path}/{new_path}")
#             continue
#         # check if new path exists, if so, skip to next journal
#         meta_data_df = scrape_pages(driver, page_url, new_path, volume_name, journal_name)
#         meta_data_dfs.append(meta_data_df)
driver.close()
pd.concat(meta_data_dfs).to_excel('trial meta data.xlsx')

In [ ]:
####### get individual journal#####

In [81]:
meta_data_dfs = []

In [17]:
#open base url
chrome = webdriver.ChromeOptions()
driver = webdriver.Chrome(driver_path, options=chrome) #ChromeDriverManager().install()
driver.get(base_url) #go to journal homepage, get title and create directory   
journals = driver.find_elements_by_class_name('title')


In [21]:
journals[11].text

'Mihr'

In [99]:
journal = journals[-3]
if len(journal.text.split('\n'))>1:
    journal_name = journal.text.split('\n')[1]
journal_name = journal.text
#journal_name = journal_name.replace(':','-')
journal_name = journal_name.translate(translate_dict).rstrip(".").strip()
journal.click()
time.sleep(3)
driver.find_element_by_class_name('imgLink').click()
driver.find_element_by_id('open-pageviewStructure-tree').click()
time.sleep(3)
volume_links = [volume.find_element_by_class_name('pagelink').get_attribute('href') for volume in driver.find_elements_by_class_name('journal-volume')]
volume_names = [volume.find_element_by_class_name('pagelink').text for volume in driver.find_elements_by_class_name('journal-volume')]
for i in range(len(volume_links)):
    volume_name = volume_names[i].translate(translate_dict).strip()
    if len(volume_name.split('\n'))>1:
        volume_name = volume_name.split('\n')[1]
    new_path = f"{journal_name}/{volume_name}"
    # check if new path exists, if so, skip to next volume
    try:
        os.makedirs(f"{folder_path}/{new_path}", exist_ok=False)
        print(f"creating folder: {folder_path}/{new_path}")
    except:
        print(f"this folder already exists: {folder_path}/{new_path}")
        pass
    driver.get(volume_links[i])
    meta_data_df = check_pages(driver, page_url, new_path, volume_name, journal_name)
    meta_data_dfs.append(meta_data_df)

In [12]:

pd.concat(meta_data_dfs).to_excel('meta-data tarbiyat.xlsx')

In [91]:
volume_names = [volume.find_element_by_class_name('pagelink').text for volume in driver.find_elements_by_class_name('journal-volume')]

volume_names

['1 \n1. Jahrgang (1314-1315hq)',
 '2. Jahrgang (1315-1316hq)',
 '3. Jahrgang (1316-1317hq)',
 '4. Jahrgang (1317-1318hq)',
 '5. Jahrgang (1318-1320hq)',
 '6. Jahrgang (1320-1321hq)',
 '7. Jahrgang (1322-1323hq)',
 '8. Jahrgang (1323hq)',
 '9. Jahrgang (1324-1325hq)']

In [92]:
volume_links = [volume.find_element_by_class_name('pagelink').get_attribute('href') for volume in driver.find_elements_by_class_name('journal-volume')]
volume_links

['https://digitale-sammlungen.ulb.uni-bonn.de/ulbbnioa/periodical/pageview/3770015',
 'https://digitale-sammlungen.ulb.uni-bonn.de/ulbbnioa/periodical/pageview/3795099',
 'https://digitale-sammlungen.ulb.uni-bonn.de/ulbbnioa/periodical/pageview/4147777',
 'https://digitale-sammlungen.ulb.uni-bonn.de/ulbbnioa/periodical/pageview/3795106',
 'https://digitale-sammlungen.ulb.uni-bonn.de/ulbbnioa/periodical/pageview/3794970',
 'https://digitale-sammlungen.ulb.uni-bonn.de/ulbbnioa/periodical/pageview/3985161',
 'https://digitale-sammlungen.ulb.uni-bonn.de/ulbbnioa/periodical/pageview/3985163',
 'https://digitale-sammlungen.ulb.uni-bonn.de/ulbbnioa/periodical/pageview/3985165',
 'https://digitale-sammlungen.ulb.uni-bonn.de/ulbbnioa/periodical/pageview/3985173']

In [13]:
driver.close()

In [ ]:
##########get metadata

In [24]:
def check_pages(driver, page_url, new_path, volume_name, journal_name):
    x = "?/:"
    y = '---'
    translate_dict = str.maketrans(x, y)    
    page_numbers = []
    page_names = []
    page_links = []
    page_mag = []
    page_volume = []
    ls = driver.find_element_by_id('goToPage').find_elements_by_tag_name('option')
    for i in ls:
        page_numbers.append(i.get_attribute(name='value'))
        page_names.append(i.text.split()[-1])
    #for each page, open it, get the name of the magazine, and then download the full image
    try:
        for ind in range(len(ls)):
            page_link = f'{page_url}/{page_numbers[ind]}'
            driver.get(page_link)
            page_links.append(page_link)
            time.sleep(3)
            temp_path = f"{journal_name}/{volume_name}"
            if len(driver.find_elements_by_class_name('npNext'))>2:# there is at least one sub-volume
                for element in driver.find_elements_by_xpath("//*[@id='navPath']/div/a[contains(@href, 'pageview')]")[2:]:
                    sub_volume = element.text.translate(translate_dict).rstrip(".").strip()
                    page_volume.append(f"{volume_name}/{sub_volume}")
                    temp_path = f"{temp_path}/{sub_volume}"
            else:
                page_volume.append(f"{volume_name}")
            mag_name = driver.find_element_by_id('navPath').find_elements_by_xpath('div')[-2].text.translate(translate_dict).rstrip(".").strip()        
            if len(mag_name.split('\n'))>1:
                mag_name = mag_name.split('\n')[1]
            page_mag.append(mag_name)
            temp_path = f"{temp_path}/{mag_name}"
            if os.path.isfile(f"{temp_path}/{page_names[ind]}.jpg"): # the file has already been downloaded
                #print(f'the file already exists: {temp_path}/{page_names[ind]}.jpg')
                continue
            if not os.path.exists(f"{folder_path}/{temp_path}"):
                os.makedirs(f"{folder_path}/{temp_path}")
                print(f"creating folder: {folder_path}/{temp_path}")
            else:
                new_url = driver.find_element_by_class_name('singlePageImage').get_attribute(name='src')
                urllib.request.urlretrieve(new_url,f"{temp_path}/{page_names[ind]}.jpg")
                print(f'downloading: {temp_path}/{page_names[ind]}.jpg')
    except Exception as e:
        print(f'exception occured in check_pages: {e}')
        return pd.DataFrame({'journal':journal_name,'volume':page_volume,'mag':page_mag,'page':page_names,'link':page_links,'extraction date':datetime.date.today()})
    return pd.DataFrame({'journal':journal_name,'volume':page_volume,'mag':page_mag,'page':page_names,'link':page_links,'extraction date':datetime.date.today()})


In [25]:
#get metadata for all journals on a base_url page, if a page hasn't been downloaded fix it

#open base url
chrome = webdriver.ChromeOptions()
driver = webdriver.Chrome(driver_path, options=chrome) #ChromeDriverManager().install()
driver.get(base_url) 
#get list of all journal links
meta_data_dfs = []
journals = driver.find_elements_by_class_name('title')
journal_count = len(journals)
x = "?/:\\"
y = '----'
translate_dict = str.maketrans(x, y)
try:
    for count in range(journal_count)[11:12]:   
        driver.get(base_url) 
        #go to journal homepage, get title and create directory   
        journals = driver.find_elements_by_class_name('title')
        journal = journals[count]
        if len(journal.text.split('\n'))>1:
            journal_name = journal.text.split('\n')[1]
        journal_name = journal.text
        journal_name = journal_name.translate(translate_dict).rstrip(".").strip()
        journal.click()
        time.sleep(3)
        print(f'checking journal: {journal_name}')
        volume_elements = [element for element in driver.find_elements_by_class_name('childLink')]
        if len(volume_elements)>0: #there are multiple volumns
            volume_links = [element.get_attribute(name='href') for element in driver.find_elements_by_class_name('childLink')]
            volume_names = [element.text for element in driver.find_elements_by_class_name('partInfo')]
            for i in range(len(volume_links)):
                volume_name = volume_names[i].translate(translate_dict).rstrip(".").strip()
                if len(volume_name.split('\n'))>1:
                    volume_name = volume_name.split('\n')[1]
                new_path = f"{journal_name}/{volume_name}"
                # check if new path exists
                if not os.path.exists(f"{folder_path}/{new_path}"):
                    os.makedirs(f"{folder_path}/{new_path}")
                    print(f"creating folder: {folder_path}/{new_path}")
                driver.get(volume_links[i])
                driver.find_element_by_class_name('imgLink').click()           
                meta_data_df = check_pages(driver, page_url, new_path, volume_name, journal_name)
                meta_data_dfs.append(meta_data_df)

        else: #no apparent additional volumes (but there still could be hidden volumes)
            driver.find_element_by_class_name('imgLink').click()
            #get potential volume(s) name(s) from dropdown menu
            try:
                time.sleep(3)
                driver.find_element_by_id('open-pageviewStructure-tree').click()
                time.sleep(3)
            except: #try again after refreshing
                driver.refresh()
                time.sleep(3)
                driver.find_element_by_id('open-pageviewStructure-tree').click()  
            time.sleep(3)
            volume_links = [volume.find_element_by_class_name('pagelink').get_attribute('href') for volume in driver.find_elements_by_class_name('journal-volume')]
            volume_names = [volume.find_element_by_class_name('pagelink').text for volume in driver.find_elements_by_class_name('journal-volume')]
            for i in range(len(volume_links)):
                volume_name = volume_names[i].translate(translate_dict).rstrip(".").strip()
                if len(volume_name.split('\n'))>1:
                    volume_name = volume_name.split('\n')[1]
                new_path = f"{journal_name}/{volume_name}"
                # check if new path exists
                if not os.path.exists(f"{folder_path}/{new_path}"):
                    os.makedirs(f"{folder_path}/{new_path}")
                    print(f"creating folder: {folder_path}/{new_path}")
                driver.get(volume_links[i])
                meta_data_df = check_pages(driver, page_url, new_path, volume_name, journal_name)
                meta_data_dfs.append(meta_data_df)
    pd.concat(meta_data_dfs).to_excel('meta-data II.xlsx')
    driver.close()
except Exception as e:
    pd.concat(meta_data_dfs).to_excel('meta-data II.xlsx')
    print(f'An exception occurred: {str(e)}')
    driver.close()
    

checking journal: Mihr
creating folder: C:/Users/user/PycharmProjects/PersianChildhood/Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/1. Ausgabe (Ḫordād 1312 -- Mai 1933)
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/1. Ausgabe (Ḫordād 1312 -- Mai 1933)/[2].jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/1. Ausgabe (Ḫordād 1312 -- Mai 1933)/[3].jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/1. Ausgabe (Ḫordād 1312 -- Mai 1933)/[4].jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/1. Ausgabe (Ḫordād 1312 -- Mai 1933)/2.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/1. Ausgabe (Ḫordād 1312 -- Mai 1933)/3.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/1. Ausgabe (Ḫordād 1312 -- Mai 1933)/4.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/1. Ausgabe (Ḫordād 1312 -- Mai 1933)/5.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/1. Ausgabe (Ḫordād 1312 -- Mai 1933)/6.jpg
downloading: Mihr/1. Jahrg

downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/1. Ausgabe (Ḫordād 1312 -- Mai 1933)/79.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/1. Ausgabe (Ḫordād 1312 -- Mai 1933)/80.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/1. Ausgabe (Ḫordād 1312 -- Mai 1933)/[84].jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/1. Ausgabe (Ḫordād 1312 -- Mai 1933)/[85].jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/2. Ausgabe (Tir 1312 -- Juni 1933)/[86].jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/2. Ausgabe (Tir 1312 -- Juni 1933)/[87].jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/2. Ausgabe (Tir 1312 -- Juni 1933)/[88].jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/2. Ausgabe (Tir 1312 -- Juni 1933)/82.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/2. Ausgabe (Tir 1312 -- Juni 1933)/83.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/2. Ausgabe (Tir 1312 -- Juni 1933)

downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/3. Ausgabe (Mordād 1312 -- Juni 1933)/[222].jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/3. Ausgabe (Mordād 1312 -- Juni 1933)/211.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/3. Ausgabe (Mordād 1312 -- Juni 1933)/212.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/3. Ausgabe (Mordād 1312 -- Juni 1933)/213.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/3. Ausgabe (Mordād 1312 -- Juni 1933)/214.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/3. Ausgabe (Mordād 1312 -- Juni 1933)/[227].jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/3. Ausgabe (Mordād 1312 -- Juni 1933)/216.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/3. Ausgabe (Mordād 1312 -- Juni 1933)/[229].jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/3. Ausgabe (Mordād 1312 -- Juni 1933)/218.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/3. Ausgabe

downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/4. Ausgabe (Šahrivar 1312 -- August 1933)/282.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/4. Ausgabe (Šahrivar 1312 -- August 1933)/283.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/4. Ausgabe (Šahrivar 1312 -- August 1933)/284.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/4. Ausgabe (Šahrivar 1312 -- August 1933)/285.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/4. Ausgabe (Šahrivar 1312 -- August 1933)/[304].jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/4. Ausgabe (Šahrivar 1312 -- August 1933)/287.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/4. Ausgabe (Šahrivar 1312 -- August 1933)/[306].jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/4. Ausgabe (Šahrivar 1312 -- August 1933)/289.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/4. Ausgabe (Šahrivar 1312 -- August 1933)/290.jpg
downloading: Mihr/1. Jahrgang (13

downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/5. Ausgabe (Mehr 1312)/363.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/5. Ausgabe (Mehr 1312)/364.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/5. Ausgabe (Mehr 1312)/365.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/5. Ausgabe (Mehr 1312)/[387].jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/5. Ausgabe (Mehr 1312)/367.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/5. Ausgabe (Mehr 1312)/368.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/5. Ausgabe (Mehr 1312)/[390].jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/5. Ausgabe (Mehr 1312)/370.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/5. Ausgabe (Mehr 1312)/[392].jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/5. Ausgabe (Mehr 1312)/371.jpg
downloading: Mihr/1. Jahrgang (1312-1313hš -- 1933-1934)/5. Ausgabe (Mehr 1312)/372.jpg
downloading: Mihr/1. Jahrg

...

downloading: Mihr/3. Jahrgang (1314-1315hš)/8. Ausgabe (Dey 1314)/[1030].jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/8. Ausgabe (Dey 1314)/[1031].jpg
creating folder: C:/Users/user/PycharmProjects/PersianChildhood/Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/[1033].jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/[1034].jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/[1035].jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/[1036].jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/866.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/867.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/868.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/869.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/870.jpg
downloading: Mihr/3. 

downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/963.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/964.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/965.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/[1138].jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/967.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/968.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/969.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/970.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/971.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/[1144].jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/[1145].jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/9. Ausgabe (Bahman 1314)/[1146].jpg
creating folder: C:/Users/user/PycharmProjects/PersianChildhood/

downloading: Mihr/3. Jahrgang (1314-1315hš)/10. Ausgabe (Esfand 1314)/1060.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/10. Ausgabe (Esfand 1314)/1061.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/10. Ausgabe (Esfand 1314)/1062.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/10. Ausgabe (Esfand 1314)/[1242].jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/10. Ausgabe (Esfand 1314)/1064.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/10. Ausgabe (Esfand 1314)/1065.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/10. Ausgabe (Esfand 1314)/[1245].jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/10. Ausgabe (Esfand 1314)/1067.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/10. Ausgabe (Esfand 1314)/1068.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/10. Ausgabe (Esfand 1314)/1069.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/10. Ausgabe (Esfand 1314)/1070.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/10. Ausgabe (Esfand 1314)/[1250].jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/10

downloading: Mihr/3. Jahrgang (1314-1315hš)/11. Ausgabe (Farvardin 1315)/1156.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/11. Ausgabe (Farvardin 1315)/1157.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/11. Ausgabe (Farvardin 1315)/1158.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/11. Ausgabe (Farvardin 1315)/1159.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/11. Ausgabe (Farvardin 1315)/[1343].jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/11. Ausgabe (Farvardin 1315)/1161.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/11. Ausgabe (Farvardin 1315)/1162.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/11. Ausgabe (Farvardin 1315)/[1346].jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/11. Ausgabe (Farvardin 1315)/1164.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/11. Ausgabe (Farvardin 1315)/1165.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/11. Ausgabe (Farvardin 1315)/1166.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/11. Ausgabe (Farvardin 1315)/1167.jpg
downloading:

downloading: Mihr/3. Jahrgang (1314-1315hš)/12. Ausgabe (Ordibehešt 1315)/1248.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/12. Ausgabe (Ordibehešt 1315)/1249.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/12. Ausgabe (Ordibehešt 1315)/1250.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/12. Ausgabe (Ordibehešt 1315)/[1441].jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/12. Ausgabe (Ordibehešt 1315)/1252.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/12. Ausgabe (Ordibehešt 1315)/[1443].jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/12. Ausgabe (Ordibehešt 1315)/1254.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/12. Ausgabe (Ordibehešt 1315)/1255.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/12. Ausgabe (Ordibehešt 1315)/1256.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/12. Ausgabe (Ordibehešt 1315)/[1447].jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/12. Ausgabe (Ordibehešt 1315)/1258.jpg
downloading: Mihr/3. Jahrgang (1314-1315hš)/12. Ausgabe (Ordibehešt 1315)/1259.jp

In [7]:
volume_links

[]

In [18]:
journal_name

'Šarq'

In [ ]:
#check yadgar and nama i-heftegi

In [9]:
#get all journals on a base_url page

#open base url
chrome = webdriver.ChromeOptions()
driver = webdriver.Chrome(driver_path, options=chrome) #ChromeDriverManager().install()
temp_url = "https://digitale-sammlungen.ulb.uni-bonn.de/ulbbnioa/periodical/titleinfo/7087683"
driver.get(temp_url) 
driver.find_element_by_class_name('imgLink').click()

#sub_volume_elements = [element for element in driver.find_elements_by_class_name('childLink')]


In [87]:
page_numbers = []
page_names = []
ls = driver.find_element_by_id('goToPage').find_elements_by_tag_name('option')
for i in ls:
    page_numbers.append(i.get_attribute(name='value'))
    page_names.append(i.text.split()[-1])
#for each page, open it, get the name of the magazine, and then download the full image
for ind in range(len(ls)):
    driver.get(f'{page_url}/{page_numbers[ind]}')
    time.sleep(3)
    new_url = driver.find_element_by_class_name('singlePageImage').get_attribute(name='src')
    mag_name = driver.find_element_by_id('navPath').find_elements_by_xpath('div')[2].text
    mag_name = mag_name.replace('/','-') #remove unformatted char
    #for a given page, go to its full image
    os.makedirs(f"{new_path}/{mag_name}", exist_ok = True)
    urllib.request.urlretrieve(new_url,f"{new_path}/{mag_name}/{page_names[ind]}.jpg")

In [88]:
driver.close()